In [10]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.6 MB/s eta 0:00:00


get from: [Open-Meteo](https://open-meteo.com/en/docs/historical-weather-api#latitude=5.0689&longitude=-75.5174&start_date=2022-10-25&end_date=2024-10-25&hourly=temperature_2m,relative_humidity_2m&timezone=auto)

In [52]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 5.0689,
	"longitude": -75.5174,
	"start_date": "2022-10-25",
	"end_date": "2024-10-25",
	"hourly": ["temperature_2m", "relative_humidity_2m"],
	"timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 5.09666109085083°N -75.445556640625°E
Elevation 2120.0 m asl
Timezone b'America/Bogota' b'-05'
Timezone difference to GMT+0 -18000 s
                           date  temperature_2m  relative_humidity_2m
0     2022-10-25 05:00:00+00:00         14.0855             95.554092
1     2022-10-25 06:00:00+00:00         14.1355             97.755669
2     2022-10-25 07:00:00+00:00         14.0355             96.490524
3     2022-10-25 08:00:00+00:00         15.0855             85.858459
4     2022-10-25 09:00:00+00:00         14.6355             88.963516
...                         ...             ...                   ...
17563 2024-10-26 00:00:00+00:00         15.6355             85.355812
17564 2024-10-26 01:00:00+00:00         16.5355             81.120453
17565 2024-10-26 02:00:00+00:00         16.5355             81.385269
17566 2024-10-26 03:00:00+00:00         15.1855             88.429756
17567 2024-10-26 04:00:00+00:00         14.2355             92.192665

[17568 rows x 

get from: [Open-Meteo](https://open-meteo.com/en/docs/air-quality-api#latitude=5.0689&longitude=-75.5174&hourly=pm10,pm2_5,carbon_monoxide,carbon_dioxide&timezone=auto&start_date=2022-10-25&end_date=2024-10-25&time_mode=time_interval)

In [53]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
	"latitude": 5.0689,
	"longitude": -75.5174,
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "carbon_dioxide"],
	"timezone": "auto",
	"start_date": "2022-10-25",
	"end_date": "2024-10-25"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
hourly_carbon_dioxide = hourly.Variables(3).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
hourly_data["carbon_dioxide"] = hourly_carbon_dioxide

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 5.099998474121094°N -75.5°E
Elevation 2120.0 m asl
Timezone b'America/Bogota' b'-05'
Timezone difference to GMT+0 -18000 s
                           date       pm10  pm2_5  carbon_monoxide  \
0     2022-10-25 05:00:00+00:00  20.799999   14.6            333.0   
1     2022-10-25 06:00:00+00:00  20.299999   14.1            316.0   
2     2022-10-25 07:00:00+00:00  20.600000   14.4            311.0   
3     2022-10-25 08:00:00+00:00  22.100000   15.5            309.0   
4     2022-10-25 09:00:00+00:00  23.600000   16.5            315.0   
...                         ...        ...    ...              ...   
17563 2024-10-26 00:00:00+00:00  19.299999   13.5           1162.0   
17564 2024-10-26 01:00:00+00:00  19.400000   13.6           1009.0   
17565 2024-10-26 02:00:00+00:00  19.299999   13.5            796.0   
17566 2024-10-26 03:00:00+00:00  18.700001   13.1            612.0   
17567 2024-10-26 04:00:00+00:00  18.500000   12.9            488.0   

       carbon_dioxide  
